<a href="https://colab.research.google.com/github/luisr96/Delivery-Routing-for-BreakfastFiesta/blob/main/shortestdistancealgo_realistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import urllib.request
import requests
import json
import pandas as pd
import random
import csv

# Leaving mine here for demonstration but it's free to get and use
bingMapsKey="Au5hDbyQjjSaQY7L2mARJHMG1_922wzhfXmSaytIdku_AvBI3pmyY98tTp-imkSf"

def goTo(startlat, startlon,endlat, endlon):
	'''Uses Bing Maps API to go from
	startlat,startlon to endlat,endlon
	Traffic info is not used'''
	
	payload = {
    "origins": [{"latitude": startlat, "longitude": startlon}],
    "destinations": [{"latitude": endlat, "longitude": endlon}],
    "travelMode": "driving",
	}

	keydict = {"key": bingMapsKey}

	r = requests.post('https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix', data = json.dumps(payload), params = keydict)

	data = (r.json())

	distance = data['resourceSets'][0]['resources'][0]['results'][0]['travelDistance'] / 1.60934 #convert from km to miles

	duration = data['resourceSets'][0]['resources'][0]['results'][0]['travelDuration']

	print(f"\nFrom: {startlat},{startlon }\nTo: {endlat},{endlon}\nDistance: {distance:.2f} miles\nDuration: {duration:.2f} minutes")

	return distance, duration


# import file
df_deliveries = pd.read_csv(f"BF_OrderLog_geocodio.csv")

# make a dataframe with all rows, and only the following columns:
df_deliveries = df_deliveries.loc[:,['ID', 'Client','Latitude','Longitude']]

# remove the pandas index column and set it to the ID
df_deliveries.set_index('ID', inplace=True)

# init and empty df to put locations in later
df_route = pd.DataFrame(columns=['ID', 'Client','Latitude','Longitude'])

# coordinates of start location
homelat = 25.86288760266286 
homelon= -80.24720464588125

# df of start location
df_home = pd.read_csv('home.csv')
df_home.set_index('ID', inplace=True)

# initialize empty lists
distancecolumn = []
timecolumn = []


# iterizer
startlat = homelat
startlon = homelon

space = 6
space_counter = 0


# while there are still deliveries to be made
while len(df_deliveries.index) != 0:

  # if car is not empty
	if space_counter != space:

		for i in df_deliveries.index: # for as many rows as there are

      # find the distances from current location to every point
			distance, duration = goTo(startlat, startlon, df_deliveries.at[i, 'Latitude'], df_deliveries.at[i, 'Longitude'])

	  	# create distance/duration data
			distancecolumn.append(distance)
			timecolumn.append(duration)


		# create columns from the distance/duration data above
		df_deliveries['Distance_from_Last'] = distancecolumn
		df_deliveries['Time_from_Last'] = timecolumn
		print(df_deliveries)


		# find the closest address
		df_closest = df_deliveries[df_deliveries.Time_from_Last==df_deliveries.Time_from_Last.min()]
		print("\nClosest point: \n")
		print(df_closest)


		# closest address's latitude
		closestlat = df_closest['Latitude'].values[0]
		# closest address's longitude
		closestlon = df_closest['Longitude'].values[0]


		# "Go" there and save the data
		distance, duration = goTo(startlat, startlon, closestlat, closestlon)

		# add closest address to route df (was initially empty)
		df_route = df_route.append(df_closest)


		print("\nRoute so far: \n")
		print(df_route)


		# delete min row so we don't go to the same place twice
		df_deliveries = df_deliveries.loc[df_deliveries['Time_from_Last']!=df_deliveries['Time_from_Last'].min()]


		# new start is the last traveled location
		startlat = closestlat
		startlon = closestlon


		# reset these to make new columns
		distancecolumn = []
		timecolumn = []

		space_counter += 1

		continue

	else: #if car is empty

    # go from wherever you are back to home
		distance, duration = goTo(startlat, startlon, homelat, homelon)

    # make the 1st row in dataframe the distance and time above
    # (we don't want to save these values, just account for them in the route)
		df_home.loc[0,'Distance_from_Last'] = distance
		df_home.loc[0,'Time_from_Last'] = duration

    # new start is home
		startlat = homelat
		startlon = homelon

    # add going back home to the route
		df_route = df_route.append(df_home)

    # "stock up" the car
		space_counter = 0

		continue

	space_counter += 1


# set the index as the 'ID' for minimum dataframe
df_route['ID'] = df_route.index
df_route.set_index('ID', inplace=True)


print("\nRoute: \n")
print(df_route)

df_route.to_csv('route_realistic.csv')

Streaming output truncated to the last 5000 lines.
Distance: 30.59 miles
Duration: 37.42 minutes

From: 25.929776,-80.28321600000001
To: 25.621769,-80.420698
Distance: 27.79 miles
Duration: 34.50 minutes

From: 25.929776,-80.28321600000001
To: 25.700156,-80.417381
Distance: 23.83 miles
Duration: 32.28 minutes

From: 25.929776,-80.28321600000001
To: 25.720914,-80.407891
Distance: 21.73 miles
Duration: 27.58 minutes

From: 25.929776,-80.28321600000001
To: 25.988207,-80.315276
Distance: 5.89 miles
Duration: 14.75 minutes

From: 25.929776,-80.28321600000001
To: 25.702964,-80.39980600000001
Distance: 22.54 miles
Duration: 29.20 minutes

From: 25.929776,-80.28321600000001
To: 25.736043,-80.393279
Distance: 20.66 miles
Duration: 25.48 minutes

From: 25.929776,-80.28321600000001
To: 25.770551,-80.251593
Distance: 18.08 miles
Duration: 26.13 minutes

From: 25.929776,-80.28321600000001
To: 25.73555,-80.385851
Distance: 20.62 miles
Duration: 25.45 minutes

From: 25.929776,-80.28321600000001
To: 2